# **GRADED CHALLENGE - 3**


---



*   Andik Al Fauzi - FTDS-RMT-018



# Problem Statement Background

---




Thelook Ecommerce adalah salah Store dengan salah satu penjualan yang cukup baik, maka dengan ini saya sebagai Data Analysis di thelook akan memberikan beberapa penjabaran terkait data-data yang sudah ada selama tahun 2022 di Quartal 1 sampai Quartal 3.
Untuk penjabaran ini saya menggunakan metode SMART.
1. Specific
  - Apakah total user yang bertransaksi berpengaruh terhadap total penjualan selama Q1, Q2 dan Q3?
  - Solusi apa yang akan digunakan untuk bisa menaikan penjualan jika ada bulan dengan total user yang lebih rendah?
2. Measurable
  - Apakah nilai user, transaksi dan penjualan yang terjadi dimasing-masing quartal bisa dihitung dengan akurat?
  - Berikan nilai persentasenya! 
3. Achievable
  - Apakah nilai penjualan membaik dari Q1 ke Q3. 
  - Berikan pendapatmu
4. Relevant
  - Apakah data yang disajikan relevant?
  - mengapa dan dengan metode apa datanya bisa relevant?
5. Time-bond Goals
  - berikan prediksi total penjualan untuk awal Quartal 4 (October 2022)?

# Objective

---



Ini adalah tugas untuk Graded Challenge 3 yang datanya diambil dari *Google Big Query* dengan dataset `thelook_ecommerce` yang rentang datanya diambil hanya selama tahun 2022 sampai bulan September.

# Data & Library Preparation

---



In [4]:
# Importing Library

import numpy as np
import pandas as pd

In [5]:
# Setting up Authentication for Google Cloud and Colab

from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
project_id = "abiding-state-376306"
client = bigquery.Client(project=project_id)

# Question Mark 1

---





1.   Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?
2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?
3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?
4. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (status transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!
5. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)

# Answer of Question Mark 1 - 1 

---



- Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [6]:
# SQL Query find total transaction base on month

totalTransactionMonth = client.query("""
SELECT
  EXTRACT(month FROM orit.created_at) AS Month,
  EXTRACT(quarter FROM orit.created_at) AS Quarter,
  COUNT(orit.created_at) AS totalTransaction
FROM bigquery-public-data.thelook_ecommerce.inventory_items AS initem
JOIN bigquery-public-data.thelook_ecommerce.order_items AS orit
ON initem.product_id = orit.product_id
WHERE orit.status = 'Complete' AND 
  EXTRACT(year from orit.created_at) = 2022 AND
  EXTRACT(quarter from orit.created_at) BETWEEN 1 AND 3
GROUP BY Month, quarter
ORDER BY Month
""").to_dataframe()

# Query to find the percentage of total Transaction by Month
sumTransaction = totalTransactionMonth['totalTransaction'].sum()

# Add new Column percentage to total transaction by month
totalTransactionMonth['Percentage'] = (totalTransactionMonth['totalTransaction'] / sumTransaction) * 100

# print dataframe
totalTransactionMonth

,Month,Quarter,totalTransaction,Percentage
0,1,1,23994,8.771949
1,2,1,22091,8.076233
2,3,1,29323,10.720174
3,4,2,25496,9.321064
4,5,2,30863,11.283182
5,6,2,30379,11.106237
6,7,3,34815,12.727991
7,8,3,37621,13.753834
8,9,3,38949,14.239337


In [7]:
# SQL Query to find total transaction base on querter

totalTransactionQuarter = client.query("""
SELECT
  EXTRACT(quarter FROM orit.created_at) AS Quarter,
  COUNT(orit.created_at) AS totalTransaction
FROM bigquery-public-data.thelook_ecommerce.inventory_items AS initem
JOIN bigquery-public-data.thelook_ecommerce.order_items AS orit
ON initem.product_id = orit.product_id
WHERE orit.status = 'Complete' AND 
  EXTRACT(year from orit.created_at) = 2022 AND
  EXTRACT(quarter from orit.created_at) BETWEEN 1 AND 3
GROUP BY quarter
ORDER BY quarter
""").to_dataframe()

# Query to find the percentage of total Transaction by Quarter
sumTransaction = totalTransactionQuarter['totalTransaction'].sum()

# Adding new column percentage
totalTransactionQuarter['Percentage'] = (totalTransactionQuarter['totalTransaction'] / sumTransaction) * 100

# print dataframe
totalTransactionQuarter

,Quarter,totalTransaction,Percentage
0,1,75408,27.568356
1,2,86738,31.710483
2,3,111385,40.721161


- Insight : 


---

- Berdasarkan data diatas, pada table pertama untuk masing-masing transaksi berada dalam kisaran nilai yang tidak terlalu jauh untuk masing-masing bulannya. Sedangkan pada table kedua terjadi peningkatan transaksi untuk masing-masing quartal yang mana kenaikan total transaksi tertinggi ada di Quarter 1 menuju Quarter 2 dengan total kenaikan sebanyak `17.622` transaksi, sementara saat menuju ke quarter 3 hanya terjadi kenaikan sebanyak `17.330`,  walaupun terjadi penurunan total transaksi tidak tidak terlalu signifikan. Dan dilihat dari percentasenya pun ada kenaikan dari masing-masing data bulanan ataupun data per quartal.

# Answer of Question Mark 1 - 2 

---



2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

In [8]:
# Query SQL to get totalSales per month

totalSalesMonth = client.query('''
SELECT 
  Month, Quarter, 
  round(SUM(sale_price)) AS totalSales
FROM ( SELECT *, 
    EXTRACT(month FROM ordItems.created_at) AS Month,
    EXTRACT(quarter FROM ordItems.created_at) AS Quarter
    FROM bigquery-public-data.thelook_ecommerce.inventory_items AS inItems
    JOIN bigquery-public-data.thelook_ecommerce.order_items AS ordItems
    ON ordItems.product_id = inItems.product_id
    WHERE ordItems.status = 'Complete' AND 
    EXTRACT(year FROM ordItems.created_at) = 2022 AND
    EXTRACT(quarter FROM ordItems.created_at) BETWEEN 1 AND 3 
    ) 
GROUP BY Month, Quarter
ORDER BY Month
''').to_dataframe()

# Query to find percentage base on total sales by month
sumSales = totalSalesMonth['totalSales'].sum()

# Define new column percentage
totalSalesMonth['Percentage'] = (totalSalesMonth['totalSales'] / sumSales) * 100

# print dataframe
totalSalesMonth

,Month,Quarter,totalSales,Percentage
0,1,1,1422483.0,8.868695
1,2,1,1194672.0,7.448371
2,3,1,1745947.0,10.885383
3,4,2,1429762.0,8.914077
4,5,2,1865714.0,11.632089
5,6,2,1771169.0,11.042633
6,7,3,2112767.0,13.172380
7,8,3,2177100.0,13.573474
8,9,3,2319758.0,14.462898


In [9]:
# Query SQL to get summary of totalSales per quarter

totalSalesQuarter = client.query('''
SELECT 
  Quarter, 
  round(SUM(sale_price)) AS totalSales
FROM ( SELECT *, 
    EXTRACT(quarter FROM ordItems.created_at) AS Quarter
FROM bigquery-public-data.thelook_ecommerce.inventory_items AS inItems
JOIN bigquery-public-data.thelook_ecommerce.order_items AS ordItems
ON ordItems.product_id = inItems.product_id
WHERE ordItems.status = 'Complete' AND 
    EXTRACT(year FROM ordItems.created_at) = 2022 AND
    EXTRACT(quarter FROM ordItems.created_at) BETWEEN 1 AND 3 
  ) 
GROUP BY Quarter
ORDER BY Quarter
''').to_dataframe()

# Query to find percentage base on total sales by quarter
sumSales = totalSalesQuarter['totalSales'].sum()

# Define new column percentage
totalSalesQuarter['Percentage'] = (totalSalesQuarter['totalSales'] / sumSales) * 100

# print dataframe
totalSalesQuarter

,Quarter,totalSales,Percentage
0,1,4363103.0,27.202455
1,2,5066644.0,31.588793
2,3,6609625.0,41.208752


- Insight : 


---


- Berdasarkan data perhitungan diatas rata-rata total penjualan per bulan mengalami kenaikan walaupun di bulan February 2022 mengalami penurunan, namun naik kembali di bulan Maret 2022. Untuk insight berdasarkan Quartal maka rata2 mengalami kenaikan presentase sebanyak `6,95%` dari Quartal 1 ke Quartal 2 dan sebanyak `7,58%` di Quartal 3. Dan Total Penjual sampai dengan Quartal 3 - 2022 senilai `US$ 16,598,765`. 

# Answer of Question Mark 1 - 3

---

3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [10]:
# Query SQL to get totalUser per month

totalUserMonth = client.query('''
SELECT
    EXTRACT(month FROM ordItems.created_at) AS Month,
    EXTRACT(quarter FROM ordItems.created_at) AS Quarter,
    COUNT(DISTINCT(ordItems.user_id)) AS TotalUser
FROM bigquery-public-data.thelook_ecommerce.inventory_items AS inItems
JOIN bigquery-public-data.thelook_ecommerce.order_items AS ordItems
ON ordItems.product_id = inItems.product_id
WHERE ordItems.status = 'Complete' AND 
    EXTRACT(year FROM ordItems.created_at) = 2022 AND
    EXTRACT(quarter FROM ordItems.created_at) BETWEEN 1 AND 3
GROUP BY Month, Quarter
ORDER BY Month
''').to_dataframe()

# Function to find percentage of total user by month
sumUser = totalUserMonth['TotalUser'].sum()

# Add new column percentage to total user by month
totalUserMonth['Percentage'] = (totalUserMonth['TotalUser'] / sumUser) * 100

# print dataframe
totalUserMonth

,Month,Quarter,TotalUser,Percentage
0,1,1,850,8.844953
1,2,1,786,8.17898
2,3,1,1009,10.49948
3,4,2,924,9.614984
4,5,2,1066,11.092612
5,6,2,1072,11.155047
6,7,3,1248,12.986472
7,8,3,1285,13.371488
8,9,3,1370,14.255983


In [11]:
# Query SQL to get totalUser per month

totalUserQuarter = client.query('''
SELECT
    EXTRACT(quarter FROM ordItems.created_at) AS Quarter,
    COUNT(DISTINCT(ordItems.user_id)) AS TotalUser
FROM bigquery-public-data.thelook_ecommerce.inventory_items AS inItems
JOIN bigquery-public-data.thelook_ecommerce.order_items AS ordItems
ON ordItems.product_id = inItems.product_id
WHERE ordItems.status = 'Complete' AND 
    EXTRACT(year FROM ordItems.created_at) = 2022 AND
    EXTRACT(quarter FROM ordItems.created_at) BETWEEN 1 AND 3
GROUP BY Quarter
ORDER BY Quarter
''').to_dataframe()

# Function to find percentage of total user by quarter
sumUser = totalUserQuarter['TotalUser'].sum()

# Add new column percentage to total user by quarter
totalUserQuarter['Percentage'] = (totalUserQuarter['TotalUser'] / sumUser) * 100

# print dataframe
totalUserQuarter

,Quarter,TotalUser,Percentage
0,1,2576,27.645417
1,2,2975,31.927452
2,3,3767,40.42713


- Insight : 

---



- Berdasarkan query data diatas, dapat dilihat total user yang melakukan transaksi per bulan selama tahun 2022 mengalami kenaikan yang cukup baik, walaupun di bulan February 2022 mengalami penurunan namun tidak signifikan. Sedangkan perbandingan per Quartal total user di Quartal 1 < Quartal 2 < Quartal 3 dengan perbandingan yaitu `2,615` < `3,085` < `3,884` terhadap keseluruhan dari total user. Kenaikan total user sekitar `5.94%` di Quartal 2 dan di Quartal 3 sebanyak `8.33%`. Total user yang melakukan transaksi ini berbanding lurus dengan Total Penjualan dan Total Transaksi yang dilakukan di `thelook_ecommerce` ini.

# Answer of Question Mark 1 - 4

---

4. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (status transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [12]:
# Query SQL to get maximal product category sales per month

totalCategory = client.query('''
SELECT
  Month, Quarter, Category, Total
  FROM (
    SELECT *, row_number() over(
      PARTITION BY Month
      ORDER BY Total DESC
    ) AS sorted
    FROM (
      SELECT 
        EXTRACT(Month FROM ordItems.created_at) AS Month,
        EXTRACT(Quarter FROM ordItems.created_at) AS Quarter,
        inItems.product_category AS Category,
        COUNT(ordItems.product_id) AS Total
        FROM bigquery-public-data.thelook_ecommerce.inventory_items AS inItems
        JOIN bigquery-public-data.thelook_ecommerce.order_items AS ordItems
        ON ordItems.product_id = inItems.product_id
        WHERE ordItems.status = 'Complete' AND
        EXTRACT(year FROM ordItems.created_at) = 2022 AND
        EXTRACT(quarter FROM ordItems.created_at) BETWEEN 1 AND 3
        GROUP BY Month, Quarter, Category
        ORDER BY Month 
      )
  )
WHERE sorted = 1
ORDER BY Month
''').to_dataframe()

# Print dataframe
totalCategory

,Month,Quarter,Category,Total
0,1,1,Jeans,1714
1,2,1,Jeans,1544
2,3,1,Fashion Hoodies & Sweatshirts,2195
3,4,2,Intimates,1964
4,5,2,Sweaters,2298
5,6,2,Intimates,2008
6,7,3,Sleep & Lounge,2452
7,8,3,Fashion Hoodies & Sweatshirts,3025
8,9,3,Fashion Hoodies & Sweatshirts,2882


- Insight : 

---



- Berikut adalah top sales per kategori per bulan per quartal selama tahun 2022, produk kategori `Jeans` merupakan Top Sales di beberapa bulan dengan total sebanyak `12,037` produk yang terjual. Untuk kategori *`Fashion Hoodies & Sweatshirts`* menduduki peringkat yang rendah karena hanya sekali menjadi Top Sales di bulan February 2022 dengan total `1,887` kali terjual. Mungkin di Quartal 4 bagian *marketing* bisa mencari strategi lain agar produk kategori *`Fashion Hoodies & Sweatshirts`* bisa memberikan kontribusi yang lebih banyak lagi terhadap penjualan.

# Answer of Question Mark 1 -5

---



5. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)

In [13]:
# Query to define the correlation dataframe
dataCorr = client.query('''
SELECT
    EXTRACT(month FROM ordItems.created_at) AS Month,
    EXTRACT(quarter FROM ordItems.created_at) AS Quarter,
    COUNT(DISTINCT(ordItems.user_id)) AS TotalUser
FROM bigquery-public-data.thelook_ecommerce.inventory_items AS inItems
JOIN bigquery-public-data.thelook_ecommerce.order_items AS ordItems
ON ordItems.product_id = inItems.product_id
WHERE ordItems.status = 'Complete' AND 
    EXTRACT(year FROM ordItems.created_at) = 2022 AND
    EXTRACT(quarter FROM ordItems.created_at) BETWEEN 1 AND 3
GROUP BY Month, Quarter
ORDER BY Month
''').to_dataframe()

# Add new column TotalSales & TotalTransaction by table TotalSalesMonth & TotalTransactionMonth
dataCorr['TotalSales'] = totalSalesMonth['totalSales']
dataCorr['TotalTransaction'] = totalTransactionMonth['totalTransaction']

# Add value of percentage
dataCorr['PercentageToSales'] = (dataCorr['TotalSales'] / dataCorr['TotalSales'].sum() * 100).apply('{:.2f}%'.format)
dataCorr['PercentageToTransaction'] = (dataCorr['TotalTransaction'] / dataCorr['TotalTransaction'].sum() *100).apply('{:.2f}%'.format)

# Print dataframe
dataCorr

,Month,Quarter,TotalUser,TotalSales,TotalTransaction,PercentageToSales,PercentageToTransaction
0,1,1,850,1422483.0,23994,8.87%,8.77%
1,2,1,786,1194672.0,22091,7.45%,8.08%
2,3,1,1009,1745947.0,29323,10.89%,10.72%
3,4,2,924,1429762.0,25496,8.91%,9.32%
4,5,2,1066,1865714.0,30863,11.63%,11.28%
5,6,2,1072,1771169.0,30379,11.04%,11.11%
6,7,3,1248,2112767.0,34815,13.17%,12.73%
7,8,3,1285,2177100.0,37621,13.57%,13.75%
8,9,3,1370,2319758.0,38949,14.46%,14.24%


In [14]:
# Check Correlation Value
dataCorr.corr()

,Month,Quarter,TotalUser,TotalSales,TotalTransaction
Month,1.000000,0.948683,0.956249,0.911801,0.943017
Quarter,0.948683,1.000000,0.905091,0.851030,0.877404
TotalUser,0.956249,0.905091,1.000000,0.988239,0.995066
TotalSales,0.911801,0.851030,0.988239,1.000000,0.992177
TotalTransaction,0.943017,0.877404,0.995066,0.992177,1.000000


- Insight : 

---

- Dari data-data diatas dapat dijabarkan bahwa total user yang bertransaksi di setiap bulannya berbanding lurus dengan total penjualan maupun dengan total transaksinya, bisa dikatakan bahwa semakin banyak user yang bertransaksi maka semakin tinggi pula nilai penjualan dan nilai transaksinya. Perbandingan persentase user dengan sales, dibandingkan user dengan transaksipun tidak terlalu signifikan, dengan range data yang bisa dibilang hampir sama, maka korelasi keduanya pun bisa dikatakan mendekati benar. Atau bisa dengan penjabaran berikut untuk korelasinya : 
 1. Korelasi antara `Total User` dengan `Total Sales` nilainya `0.993725` yang artinya ini mendekati benar
 2. Korelasi antara `Total User` dengan `Total Transaksi` nilainya `0.993744` yang artinya pun ini benar
 
- Semakin besar nilai korelasi mendekati angka 1, maka korelasi yang terjadi diantara kedua perbandingannya mendekati benar.

# Analysis Point

---



- Berikut beberapa point penjabaran Analisis terkait data table diatas


 1. Total Transaksi yang terjadi selama Quartal 1 sampai dengan Quartal 3 sebanyak `277,324` total transaksi, dengan total terbanyak berada pada Quartal 3. meskipun total transaksi pada Quartal 1 dan Quartal 2 tidak sebanyak Quartal 3 tetapi hal ini sudah sangat baik mengingat target transaksi semakin meningkat pada setiap Quartalnya.
 2. Total Penjualan yang terjadi sampai dengan Quartal 3 senilai `US$ 16,598,765` dengan yang tertinggi berada pada Quartal 3 senilai `US$ 6,889,375`. Hal ini sangat berkaitan dengan poin 1 dimana pada Quartal 3 tersebut juga total transaksinya berada pada urutan pertama. Jika dilihat nilai per Quartalnya pun sudah sangat baik, karena terjadi kenaikan nilai transaksi pada masing-masing Quartal.
 3. Sebanyak `9,584` user sampai dengan Quartal 3 yang telah bertransaksi di `thelook_ecommerce` menandakan bahwa para pelanggan mempercayai `thelook_ecommerce` sebagai pilihan mereka dalam mencari barang yang mereka inginkan. Pencapaian ini harus terus dapat dipertahankan dan juga ditingkatkan, mengingat banyaknya pesaing pada saat ini. Quartal 3 merupakan yang tertinggi dengan total user sebanyak `40,53%` dari total keseluruhan.
 4. Category `Jeans` merupakan yang paling laris dalam kategori produk di `thelook_ecommerce`. Meraih 5 kali top sales sampai dengan Quartal 3 dengan total produk terjual sebanyak `12,037`. Hal ini bisa dikatakan bahwasanya para user senang dengan produk `Jeans` yang dijual di `thelook_ecommerce`, baik dari segi kualitas maupun harga. Sedangkan kategori `Fashion Hoodies & Sweatshirts` merupakan yang terendah, hal ini bisa terjadi karena kurangnya segi marketing strategi agar produk tersebut bisa semakin banyak terjual.
 5. Korelasi jumlah pengunjung yang datang dan bertransaksi dengan Total Transaksi yang terjadi sangat berbanding lurus. Hal ini dapat berarti baik karena `thelook_ecommerce` bisa menyediakan alternatif sarana lain agar para pengunjung semakin betah dan semakin banyak bertransaksi disini. 



# Question Point 

---


1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!
 - Ya, Problem statement yang didefinisikan diawal dapat diukur dengan jelas, karena disini saya menggunakan persentase pengukuran agar data yang disajikan dapat lebih mudah dianalisa dan dipresentasikan
 - Solusi jika di Tahun berikutnya ada periode dengan total user yang lebih rendah dari periode sebelumnya maka `thelook_ecommerce` bisa menyiasatinya dengan misal memberikan lebih banyak promo ataupun hal lainnya sehingga bisa meningkatkan traffic user kembali
2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?
 - Ya, sangat masuk akal karena berdasarkan data total sales di awal quartal 3 (*July 2022*) total salesnya mencapai `US$ 2,171,148`. Jadi kami yakin target senilai `US$ 250.000` bisa dapat tercapai di awal Quartal 4. Disini saya membandingkan dengan data di awal Quartal 3, karena saya berpendapat awal Quartal 3 akan sama dengan awal Quartal 4.
3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!
 - Menurut saya sangat layak jika kita mendapatkan investor baru di Quartal 4 sebab dalam 3 Kuartal terakhir di tahun 2022, tersaji data total User dan total Penjualan sebagai berikut : 
 >1. Quartal 1 : `2,615` user dengan nilai `US$ 4,430,018`
 >2. Quartal 2 : `3,085` user dengan nilai `US$ 5,279,372`
 >3. Quartal 3 : `3,884` user dengan nilai `US$ 6,889,375`

 Hal berikut bisa diartikan bahwa *Gross Merchandise Value (GMV)* dari `thelook_ecommerce` mengalami peningkatan yang cukup signifikan di masing-masing Quartal dengan nilai rata-rata dikisaran `US$ 1,726.39` per user per penjualan. Di Quartal 4 kami optimis akan semakin banyak customer baru yang akan bertransaksi di `thelook_ecommerce`, tanpa mengesampingkan para pelanggan setia yang selama ini terus bertransaksi di `thelook_ecommerce`.

# Problem Statement Solving

---





1. Specific
  - Data user yang bertransaksi memiliki kontribusi besar terhadap nilai transaksi dan nilai penjualan, hal ini didukung dari nilai korelasinya yang mendekati benar
  - Jika terjadi penurunan user maka `thelook_ecommerce` harus cepat bertindak dengan menggunakan *marketing strategy* yang lain, hal ini dilakukan untuk dapat menarik user baru dan mempertahankan user yang sudah ada
2. Measurable
  - Ya. Nilai user, transaksi dan penjualan dapat dihitung dengan akurat. Hal ini didapat karena database yang diberikan nilainyapun akurat.
  - Untuk persentasenya sudah disajikan dimasing-masing *Question* yang diberikan
3. Achiveable
  - Ya, total penjualan di `thelook_ecommerce` mengalami peningkatan baik dari perbandingan penjualan per bulan maupun perbandingan penjualan per kuartal, meskipun ada dibeberapa data ada penurunan, namun hal tersebut tidak terlalu signifikan. Hal tersebut didukung dari kinerja para staff `thelook_ecommerce` yang sudah sangat baik dalam menghandle pelanggan.
4. Relevant
  - Data yang tersajipun relevant karena saya berikan nilai persentase baik dimasing-masing bulan maupun per kuartalnya. Nilai persentasenya saya dapatkan dengan fungsi-fungsi yang tersedia dengan nilai akurasi yang bisa dihandalkan.
5. Time-bond Goals
 - Untuk Quartal 4 penetapan akan target yang diberikan saya bandingkan dengan target yang dicapai di Quartal sebelumnya, karena jika dilihat dari polanya, maka saya optimis target tersebut akan dapat tercapai dengan baik.
 - Untuk prediksi total penjualan di awal Quartal 4 *(October 2022)* adalah dikisaran 10,8% - 10,9% sesuai dengan presentase nilai rata-rata penjualan dibulan-bulan sebelumnya, jadi prediksi untuk total nilai penjualan di bulan *October 2022* dikisaran `US$ 25,923,651,3` tetapi hal ini masih bisa dipengaruhi oleh faktor eksternal seperti kondisi ekonomi, tren pasar, persaingan bisnis dan sebagainya.



# Conclusion

---



- Dari data-data yang telah tersaji diatas dapat disimpulkan bahwa korelasi antara total user yang sukses bertransaksi di `thelook_ecommerce` berbanding lurus dengan total transaksi dan total nilai penjualannya. Data-data yang disajikan pun sebisa mungkin dapat dipertanggungjawabkan kebenarannya. Strategi *Marketing* yang dijalankan oleh `thelook_ecommerce` pun sudah baik dan harus bisa dipertahankan agar bisa menarik lebih banyak lagi user yang akan bertransaksi di kemudian hari.